In [1]:
!pip install open_spiel pgx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.7/410.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.5 MB/s eta 0:00:00


In [2]:
import random
from tqdm import tqdm
import numpy as np
import jax
import jax.numpy as jnp
import pyspiel
import pgx
from pgx.chess import State

pgx.__version__

'1.1.0'

In [3]:
game = pyspiel.load_game('chess')
env = pgx.make("chess")
init = jax.jit(env.init)
step = jax.jit(env.step)


def check(seed):
    np.random.seed(seed)
    spiel_state = game.new_initial_state()
    pgx_state = init(jax.random.PRNGKey(0))  # seed is not related
    for _ in range(512):  # pgx chess terminates after 512 steps (following AZ paper)
        fen_before = spiel_state.debug_string().splitlines()[0][5:]

        expected_legal_actions = [m for m in spiel_state.legal_actions()]
        if not expected_legal_actions:
            break

        # check legal actions
        ok = pgx_state.legal_action_mask.sum() == len(expected_legal_actions)
        for a in expected_legal_actions:
            ok = ok and pgx_state.legal_action_mask[a]

        if not ok:
            print("legal action mask is different")
            legal_actions = jnp.nonzero(pgx_state.legal_action_mask)[0]
            pgx_state.save_svg("failed.svg")
            assert False, f"\n{fen_before}\n{pgx_state.legal_action_mask.sum()} != {len(expected_legal_actions)}\nactual:{legal_actions}\nexpected:{expected_legal_actions}"

        # step by OpenSpiel
        action = np.random.choice(expected_legal_actions)
        spiel_state.apply_action(action)
        fen_after = spiel_state.debug_string().splitlines()[0][5:]

        # step by Pgx
        pgx_state = step(pgx_state, jnp.int32(action))

        # check state transition
        assert pgx_state._to_fen() == fen_after, f"\n{fen_before}\nactual:{pgx_state._to_fen()}\nexpected: {fen_after}"


In [5]:
for i in tqdm(range(100)):
    check(i)

100%|██████████| 100/100 [53:31<00:00, 32.11s/it]
